<a href="https://colab.research.google.com/github/riyakwl28/musix/blob/master/mir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd /content/drive/My Drive

/content/drive/My Drive


In [0]:
cd music_genre


/content/drive/My Drive/music_genre


In [0]:
ls


convert-to-wav.py  img_data/  MIR/  Untitled0.ipynb


In [0]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [0]:
import matplotlib.pyplot as plt
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'/content/drive/My Drive/music_genre/MIR/genres/{g}'):
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 720x720 with 0 Axes>

In [0]:


header = 'filename chroma_stft spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()



In [0]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'/content/drive/My Drive/music_genre/MIR/genres/{g}'):
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [0]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00044.wav,0.390212,2279.124558,2375.102120,5198.360233,0.092570,-109.509285,86.922406,-8.607986,64.494557,...,11.087480,-5.085794,3.976359,-12.859741,12.343857,0.026217,-0.741570,-5.126619,3.303441,blues
1,blues.00010.wav,0.303954,1410.469096,1512.620983,2767.274933,0.062981,-173.707514,137.156400,-23.766517,26.989370,...,1.800535,-2.988374,-1.869389,-0.800155,-3.023782,-0.445196,-1.998228,-0.743599,-4.986112,blues
2,blues.00002.wav,0.363603,1552.481958,1747.165985,3040.514948,0.076301,-90.754394,140.459907,-29.109965,31.689014,...,5.789265,-8.905224,-1.083720,-9.218359,2.455805,-7.726901,-1.815724,-3.433434,-2.226821,blues
3,blues.00075.wav,0.466436,2225.216647,2255.195005,4703.188227,0.108587,-86.352338,92.675245,-15.377407,54.059777,...,12.181273,-6.097625,7.025420,-5.805348,7.432128,-6.075143,3.382991,-4.276934,1.238193,blues
4,blues.00039.wav,0.255321,1151.340441,1709.193938,2312.577683,0.035339,-301.686274,111.539209,21.895657,16.145590,...,-10.572882,-15.687018,-12.598773,-12.245661,-14.543065,-10.092300,-5.780286,-7.776049,-7.186643,blues


In [0]:
#dropping unnecessaary detials
data.shape
data = data.drop(['filename'],axis=1)

In [0]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [0]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
len(y_train),len(y_test)

(880, 220)

In [0]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [0]:
#model def


from keras import models
from keras import layers
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 680 samples, validate on 200 samples
Epoch 1/30
680/680 [==============================] - 3s 4ms/step - loss: 2.2860 - acc: 0.1647 - val_loss: 2.1631 - val_acc: 0.3600
Epoch 2/30
680/680 [==============================] - 0s 44us/step - loss: 2.1176 - acc: 0.3912 - val_loss: 2.0338 - val_acc: 0.3550
Epoch 3/30
680/680 [==============================] - 0s 43us/step - loss: 1.9533 - acc: 0.4221 - val_loss: 1.8958 - val_acc: 0.3800
Epoch 4/30
680/680 [==============================] - 0s 33us/step - loss: 1.7785 - acc: 0.4338 - val_loss: 1.7700 - val_acc: 0.3950
Epoch 5/30
680/680 [==============================] - 0s 36us/step - loss: 1.6224 - acc: 0.4735 - val_loss: 1.6605 - val_acc: 0.4400
Epoch 6/30
680/680 [==============================] - 0s 37us/step - loss: 1.4988 - acc: 0.5029 - val_loss: 1.5588 - val_acc: 0.4350
Epoch 7/30
680/680 [=================

In [0]:
results


[1.0528824437748303, 0.6227272748947144]